## Test data

In [ ]:
import utils

test_data = utils.load_lama_raw("../data/LAMA/TREx/")

In [ ]:
len(test_data)

In [ ]:
import numpy as np
import json

# topk = 2

np.random.seed(42)
n_selections = 200
selections = np.random.choice(len(test_data), (3, n_selections), replace=False).tolist()

with open("test_data_selections.json", "w") as fp:
    json.dump(selections, fp)

## Reranker candidates

In [ ]:
from datasets import load_from_disk
from tqdm import tqdm

import utils

train_data = load_from_disk("../data/HF_TREX_sentences")
    
test_data = utils.load_lama_raw("../data/LAMA/TREx")


In [ ]:
import json

with open("test_data_selections.json", "r") as fp:
    test_data_selected_indices = json.load(fp)
test_data_selected_indices = test_data_selected_indices[0]


In [ ]:
with open("../data/fact_to_ids.json", 'r') as fp:
    fact_to_ids = json.load(fp)

In [ ]:
def test_data_proponents(test_data_selections):
    proponents = []
    for rec in test_data_selections:
        predicate = rec['predicate_id']
        obj = rec['obj_uri']
        sub = rec['sub_uri']
        fact = ",".join((predicate, sub, obj))
        proponents.append(fact_to_ids[fact])
    return proponents

test_data_selections = [test_data[i] for i in test_data_selected_indices]
proponents = test_data_proponents(test_data_selections)

In [ ]:
def samples_with_same_obj_in_trex(trex, objs):
    res = {}
    for index, rec in enumerate(tqdm(trex)):
        for fact in rec['facts']:
            predicate, sub, obj = fact.split(",")
            if obj in objs:
                ids = res.setdefault(obj, set())
                ids.add(index)
    for n, ids in res.items():
        res[n] = sorted(list(ids))
    return res


target_sharing_ids = samples_with_same_obj_in_trex(train_data, [x['obj_uri'] for x in test_data])


In [ ]:
with open("target_sharing_ids.json", "w") as fp:
    json.dump(target_sharing_ids, fp)